In [ ]:
# Copyright  2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Pytorch with Ray on Vertex AI

### Configuration

In [2]:
! pip install --user -q "google-cloud-aiplatform[ray]>=1.56.0" \
                        "ray[data,train,tune,serve]==2.9.3"
                        # datasets \
                        # evaluate \
                        # accelerate \
                        # transformers \
                        # torch \
                        # numpy \
                        # pandas

In [5]:
import time
import numpy as np
import joblib
import pandas as pd
import seaborn as sns
import xgboost as xgb

# from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
# from sklearn.compose import ColumnTransformer,make_column_transformer
# from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
# from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix,confusion_matrix,classification_report
# from sklearn.pipeline import Pipeline
# from sklearn.ensemble import RandomForestClassifier

# from google.cloud import bigquery
from google.cloud import aiplatform
# from google.cloud.aiplatform.preview import vertex_ray

import ray
from ray.runtime_env import RuntimeEnv
from ray.air.config import RunConfig
from ray.air import CheckpointConfig, ScalingConfig
from ray.util.joblib import register_ray

pd.__version__, ray.__version__

('2.1.4', '2.9.3')

In [6]:
# @title Define constants
PROJECT_NBR = "721521243942"
PROJECT_ID = "ai-hangsik"
REGION="us-central1"
RAY_CLUSTER_NM = "ray293-cluster-20250217-075541"

In [7]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

In [8]:
ray.shutdown()

In [9]:

RAY_ADDRESS=f"vertex_ray://projects/{PROJECT_NBR}/locations/{REGION}/persistentResources/{RAY_CLUSTER_NM}"
print(f"RAY_ADDRESS:{RAY_ADDRESS}")

RUNTIME_ENV = {
  "pip": [
        "google-cloud-aiplatform[ray]>=1.56.0",
        "ray[data]==2.9.3",
        "ray[train]==2.9.3",
        "ray[tune]==2.9.3",
        "torch==2.1.2",
        "torchvision==0.16.2",
        "torchmetrics==1.2.1",
        "setuptools==69.5.1",
        "ipython",
  ],
}

ray.init(address=RAY_ADDRESS,runtime_env=RUNTIME_ENV)

RAY_ADDRESS:vertex_ray://projects/721521243942/locations/us-central1/persistentResources/ray293-cluster-20250217-075541
[Ray on Vertex AI]: Cluster State = State.RUNNING


SIGTERM handler is not set because current thread is not the main thread.


Python version:,3.10.14
Ray version:,2.9.3
Vertex SDK version:,1.80.0
Dashboard:,22af56cc3c3b6ab6-dot-us-central1.aiplatform-training.googleusercontent.com
Interactive Terminal Uri:,001d0c5669882547-dot-us-central1.aiplatform-training.googleusercontent.com
Cluster Name:,ray293-cluster-20250217-075541


(TunerInternal pid=8339) [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
(TunerInternal pid=8339) AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.


(TunerInternal pid=8339) 
(TunerInternal pid=8339) View detailed results here: /root/ray_results/TorchTrainer_2025-02-17_00-50-32
(TunerInternal pid=8339) To visualize your results with TensorBoard, run: `tensorboard --logdir /root/ray_results/TorchTrainer_2025-02-17_00-50-32`
(TunerInternal pid=8339) 
(TunerInternal pid=8339) Training started with configuration:
(TunerInternal pid=8339) ╭─────────────────────────────────────────────────╮
(TunerInternal pid=8339) │ Training config                                 │
(TunerInternal pid=8339) ├─────────────────────────────────────────────────┤
(TunerInternal pid=8339) │ train_loop_config/batch_size_per_worker      10 │
(TunerInternal pid=8339) │ train_loop_config/epochs                      5 │
(TunerInternal pid=8339) │ train_loop_config/lr                      0.001 │
(TunerInternal pid=8339) ╰─────────────────────────────────────────────────╯


...

(TunerInternal pid=8339) 
(TunerInternal pid=8339) Training finished iteration 3 at 2025-02-17 00:51:24. Total running time: 51s
(TunerInternal pid=8339) ╭───────────────────────────────╮
(TunerInternal pid=8339) │ Training result               │
(TunerInternal pid=8339) ├───────────────────────────────┤
(TunerInternal pid=8339) │ checkpoint_dir_name           │
(TunerInternal pid=8339) │ time_this_iter_s      11.4413 │
(TunerInternal pid=8339) │ time_total_s          45.2368 │
(TunerInternal pid=8339) │ training_iteration          3 │
(TunerInternal pid=8339) │ accuracy              0.83893 │
(TunerInternal pid=8339) │ loss                  0.43932 │
(TunerInternal pid=8339) ╰───────────────────────────────╯


Train Epoch 4:   1%|          | 22/2000 [00:00<00:09, 211.40it/s]


(TunerInternal pid=8339) 
(TunerInternal pid=8339) Training finished iteration 4 at 2025-02-17 00:51:35. Total running time: 1min 1s
(TunerInternal pid=8339) ╭───────────────────────────────╮
(TunerInternal pid=8339) │ Training result               │
(TunerInternal pid=8339) ├───────────────────────────────┤
(TunerInternal pid=8339) │ checkpoint_dir_name           │
(TunerInternal pid=8339) │ time_this_iter_s       10.604 │
(TunerInternal pid=8339) │ time_total_s          55.8408 │
(TunerInternal pid=8339) │ training_iteration          4 │
(TunerInternal pid=8339) │ accuracy              0.84823 │
(TunerInternal pid=8339) │ loss                   0.4239 │
(TunerInternal pid=8339) ╰───────────────────────────────╯


Test Epoch 4: 100%|██████████| 334/334 [00:01<00:00, 316.49it/s]


(TunerInternal pid=8339) 
(TunerInternal pid=8339) Training finished iteration 5 at 2025-02-17 00:51:45. Total running time: 1min 12s
(TunerInternal pid=8339) ╭───────────────────────────────╮
(TunerInternal pid=8339) │ Training result               │
(TunerInternal pid=8339) ├───────────────────────────────┤
(TunerInternal pid=8339) │ checkpoint_dir_name           │
(TunerInternal pid=8339) │ time_this_iter_s      10.3631 │
(TunerInternal pid=8339) │ time_total_s          66.2039 │
(TunerInternal pid=8339) │ training_iteration          5 │
(TunerInternal pid=8339) │ accuracy              0.85633 │
(TunerInternal pid=8339) │ loss                  0.40158 │
(TunerInternal pid=8339) ╰───────────────────────────────╯
(TunerInternal pid=8339) 
(TunerInternal pid=8339) Training completed after 5 iterations at 2025-02-17 00:51:47. Total running time: 1min 13s
(TunerInternal pid=8339) 


In [10]:
import os
from typing import Dict

import torch
from filelock import FileLock
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import Normalize, ToTensor
from tqdm import tqdm

import ray.train
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer

def get_dataloaders(batch_size):
    # Transform to normalize the input images
    transform = transforms.Compose([ToTensor(), Normalize((0.5,), (0.5,))])

    with FileLock(os.path.expanduser("~/data.lock")):
        # Download training data from open datasets
        training_data = datasets.FashionMNIST(
            root="~/data",
            train=True,
            download=True,
            transform=transform,
        )

        # Download test data from open datasets
        test_data = datasets.FashionMNIST(
            root="~/data",
            train=False,
            download=True,
            transform=transform,
        )

    # Create data loaders
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    return train_dataloader, test_dataloader


# Model Definition
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train_func_per_worker(config: Dict):
    lr = config["lr"]
    epochs = config["epochs"]
    batch_size = config["batch_size_per_worker"]

    # Get dataloaders inside the worker training function
    train_dataloader, test_dataloader = get_dataloaders(batch_size=batch_size)

    # [1] Prepare Dataloader for distributed training
    # Shard the datasets among workers and move batches to the correct device
    # =======================================================================
    train_dataloader = ray.train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = ray.train.torch.prepare_data_loader(test_dataloader)

    model = NeuralNetwork()

    # [2] Prepare and wrap your model with DistributedDataParallel
    # Move the model to the correct GPU/CPU device
    # ============================================================
    model = ray.train.torch.prepare_model(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    # Model training loop
    for epoch in range(epochs):
        if ray.train.get_context().get_world_size() > 1:
            # Required for the distributed sampler to shuffle properly across epochs.
            train_dataloader.sampler.set_epoch(epoch)

        model.train()
        for X, y in tqdm(train_dataloader, desc=f"Train Epoch {epoch}"):
            pred = model(X)
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        test_loss, num_correct, num_total = 0, 0, 0
        with torch.no_grad():
            for X, y in tqdm(test_dataloader, desc=f"Test Epoch {epoch}"):
                pred = model(X)
                loss = loss_fn(pred, y)

                test_loss += loss.item()
                num_total += y.shape[0]
                num_correct += (pred.argmax(1) == y).sum().item()

        test_loss /= len(test_dataloader)
        accuracy = num_correct / num_total

        # [3] Report metrics to Ray Train
        # ===============================
        ray.train.report(metrics={"loss": test_loss, "accuracy": accuracy})


def train_fashion_mnist(num_workers=3, use_gpu=False):
    global_batch_size = 32

    train_config = {
        "lr": 1e-3,
        "epochs": 5,
        "batch_size_per_worker": global_batch_size // num_workers,
    }

    # Configure computation resources
    scaling_config = ScalingConfig(num_workers=num_workers, use_gpu=use_gpu)

    # Initialize a Ray TorchTrainer
    trainer = TorchTrainer(
        train_loop_per_worker=train_func_per_worker,
        train_loop_config=train_config,
        scaling_config=scaling_config,
    )

    # [4] Start distributed training
    # Run `train_func_per_worker` on all workers
    # =============================================
    result = trainer.fit()
    print(f"Training result: {result}")


In [11]:
if __name__ == "__main__":
    train_fashion_mnist(num_workers=3, use_gpu=True)

Training result: Result(
  metrics={'loss': 0.40158225087627414, 'accuracy': 0.8563287342531494},
  path='/root/ray_results/TorchTrainer_2025-02-17_00-50-32/TorchTrainer_3156e_00000_0_2025-02-17_00-50-33',
  filesystem='local',
  checkpoint=None
)


In [12]:
ray.shutdown()